In [1]:
import pandas as pd

In [3]:
raw_data = pd.read_csv("card.csv", sep = ';')

In [4]:
data = raw_data

In [6]:
data.ftypes

date of birth    datetime64[ns]:dense
age                       int64:dense
sex                      object:dense
account                  object:dense
id                       object:dense
issued                    int64:dense
active                    int64:dense
payroll                   int64:dense
segment                  object:dense
dtype: object

In [5]:
a = raw_data[[0]].apply(pd.to_datetime)
data['date of birth'] = a['date of birth']


In [6]:
for col in ['sex', 'account', 'segment']:
    data[col] = data[col].astype('category')
    data[col] = data[col].cat.codes

In [7]:
bcp_data = data

In [8]:
iData = data.drop(data[data.issued == 0].index)

In [9]:
iData.active.unique()

array([0, 1])

In [28]:
piData = data.drop(data[data.issued == 1].index)

In [30]:
data[(data['issued'] == 1) & (data['active'] == 1)].shape

(816, 9)

In [10]:
from sklearn.cross_validation import train_test_split

/opt/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [123]:
iData.drop(iData[['active']], axis=1).shape

(5783, 8)

In [19]:
 #train_test_split(pdonations[["contbr_st", "contbr_occupation", "cand_nm"]], pdonations["contb_receipt_amt"], test_size=0.33, random_state=1)
#train, test, y_train, y_test = train_test_split(iData.drop(iData[['active', 'id']], axis=1), iData['active'], test_size=.33, random_state=1)
train, test, y_train, y_test = train_test_split(iData[[1,2,3,7,8]], iData['active'], test_size=.33, random_state=1)

In [20]:
train.head(2)

,age,sex,account,payroll,segment
20278,26,1,4,1,5
8116,49,1,2,0,5


In [13]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [38]:
from sklearn.svm import SVC

In [91]:
from sklearn.naive_bayes import GaussianNB

In [96]:
from sklearn.tree import DecisionTreeClassifier

In [39]:
model1 = SVC()

In [21]:
model = LogisticRegression(class_weight='balanced')
model.fit(train,y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [40]:
model1.fit(train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
predicted = model.predict(test)

In [23]:
print(metrics.classification_report(predicted,  y_test))
#print(metrics.confusion_matrix(y_test, predicted))

             precision    recall  f1-score   support

          0       0.65      0.91      0.76      1195
          1       0.57      0.19      0.29       714

avg / total       0.62      0.64      0.58      1909



In [25]:
import numpy as np

In [101]:
a = np.array([predicted, y_test])

In [48]:
np.unique(predicted)

array([0])

In [26]:
np.count_nonzero(predicted)

714

In [104]:
a = np.transpose(a)

In [105]:
for i in range(1909):
    if (a[i][0] == a[i][1] == 1):
        print(i)

200
447
702
829
940
1158
1254
1631


In [129]:
iData.plot.scatter(x='sex', y='active')

In [146]:
ifData = iData[iData['sex']==1]
imData = iData[iData['sex']==2]

In [157]:
imData.shape

(1514, 9)

In [151]:
infData = ifData[[1,3,5,6,7,8]]
inmData = imData[[1,3,5,6,7,8]]
#inData['age'] = iData['age']/max(iData['age'])
inmData = (inmData - inmData.mean())/(inmData.max()-inmData.min())
inmData['issued'] = imData['issued']
inmData['active'] = imData['active']
infData = (infData - infData.mean())/(infData.max()-infData.min())
infData['issued'] = ifData['issued']
infData['active'] = ifData['active']


In [171]:
infData.account.unique()

array([-0.01069978, -0.51069978, -0.17736645,  0.15596689,  0.32263355,
        0.48930022])

In [165]:
train, test, y_train, y_test = train_test_split(inmData[[0,1,4,5]], inmData['active'], test_size=.33, random_state=1)

In [167]:
model = DecisionTreeClassifier()
model.fit(train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [168]:
predicted = model.predict(test)

In [169]:
print(metrics.classification_report(predicted,  y_test))

             precision    recall  f1-score   support

          0       0.94      0.84      0.89       471
          1       0.05      0.14      0.07        29

avg / total       0.89      0.80      0.84       500



In [170]:
inData

,age,account,issued,active,payroll,segment
0,0.336280,0.010635,1,0,-0.483832,-0.497735
1,-0.005492,0.010635,1,0,-0.483832,0.302265
2,0.108432,0.010635,1,0,-0.483832,-0.497735
3,0.197040,0.010635,1,0,-0.483832,-0.497735
4,0.159065,0.010635,1,0,0.516168,-0.497735
5,0.032483,0.010635,1,0,-0.483832,0.302265
6,0.070457,0.010635,1,0,0.516168,0.302265
7,0.260331,0.010635,1,0,-0.483832,-0.497735
8,0.222356,0.010635,1,0,-0.483832,-0.497735
9,0.159065,0.010635,1,0,-0.483832,-0.497735
